In [1]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.linear_model import LogisticRegression as SklearnLogisticRegression
from sklearn.model_selection import train_test_split
from concrete.ml.sklearn import LogisticRegression as ConcreteLogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler

%matplotlib inline
import matplotlib.pyplot as plt
from IPython.display import display

In [2]:
selected_features_10 = [
    "mean compactness",
    "mean concave points",
    "radius error",
    "area error",
    "worst texture",
    "worst perimeter",
    "worst area",
    "worst smoothness",
    "worst concave points",
    "worst symmetry"
]

In [4]:
data = datasets.load_breast_cancer()
bc = pd.DataFrame(data.data, columns = data.feature_names)
bc['class'] = data.target

bc_reduced = bc[selected_features_10]
X, y = bc_reduced.to_numpy(), data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

# sc = StandardScaler()
rc = RobustScaler()

X_train = rc.fit_transform(X_train)
X_test = rc.transform(X_test)

In [15]:
logreg = SklearnLogisticRegression()
logreg.fit(X_train, y_train)
y_pred_test = np.asarray(logreg.predict(X_test))
sklearn_acc = np.sum(y_pred_test == y_test) / len(y_test) * 100

#q_logreg = ConcreteLogisticRegression(n_bits={"op_inputs": 5, "op_weights": 2})
q_logreg = ConcreteLogisticRegression(n_bits=5)
q_logreg.fit(X_train, y_train)
#q_logreg.compile(X_train)
q_y_pred_test = q_logreg.predict(X_test)
quantized_accuracy = (q_y_pred_test == y_test).mean() * 100

q_logreg.compile(X_train)
#q_y_pred_fhe = q_logreg.predict(X_test, execute_in_fhe=True)
q_y_pred_fhe = q_logreg.predict(X_test, fhe="execute")
homomorphic_accuracy = (q_y_pred_fhe == y_test).mean() * 100

print(f"Sklearn model accuracy  : {sklearn_acc:.4f}%")
print(f"Quantized model accuracy: {quantized_accuracy:.4f}%")
print(f"Homomorphic accuracy    : {homomorphic_accuracy:.4f}%")

Sklearn model accuracy  : 94.7368%
Quantized model accuracy: 93.8596%
Homomorphic accuracy    : 93.8596%
